# Tratamento de Texto

Quando você pensa em limpeza de dados, uma tarefa que provavelmente vem à mente é a manipulação de texto. Afinal, quando as pessoas inserem dados em um formulário ou diferentes convenções de formatação são anexadas, você provavelmente se verá padronizando os dados e tentando torná-los consistentes. Você também buscará valores que se perderam na tradução e estão inutilizáveis.

Nesta seção, abordaremos uma variedade de técnicas para manipular texto e executar tarefas como localizar, substituir e dividir valores. Ao longo do caminho, aprenderemos algumas expressões regulares para realizar o reconhecimento de padrões nessas tarefas.

Primeiro, vamos trazer nossas dependências e analisar este conjunto de dados do GitHub. Observe como temos algumas informações de contato, bem como um log de endereços IP de diferentes usuários. Aprenderemos como executar algumas operações de texto comuns para limpar este conjunto de dados e garantir alguma consistência.

In [ ]:
import pandas as pd 
import numpy as np 

url = 'https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/unprocessed/contacts.csv'
df = pd.read_csv(url)

df

Estas são as operações de string comuns que podemos usar no Pandas. Observe que elas normalmente aceitam uma expressão regular como padrão, e abordaremos isso.

| Função | Descrição |
|------------|-----------------------------------------------------------------------------|
| `count()` | Conta o número de instâncias em um padrão |
| `contains()` | Retorna um valor booleano True/False indicando se uma string contém um padrão |
| `replace()` | Substitui os padrões encontrados em uma string por outra string especificada. |
| `fullmatch()` | Determina se a string inteira corresponde ao padrão |
| `split()` | Divide uma string em strings separadas usando o padrão como separador |
| `extract()` | Encontra todas as ocorrências de um padrão e as agrupa em colunas |
| `findall()` | Encontra todas as ocorrências de um padrão e as agrupa em uma lista |

Mas primeiro, precisamos abordar alguns conceitos básicos sobre expressões regulares.

## Noções básicas de expressões regulares

Se você já usou curingas para pesquisar padrões de texto, expressões regulares são semelhantes. **Expressões regulares** são uma linguagem de programação especial, especificamente para a correspondência de padrões de texto complexos. Elas permitem a correspondência, divisão e substituição de texto com base em uma sintaxe de padrão padronizada. Você pode encontrá-las implementadas em centenas de plataformas, incluindo Python, Java e SQL. Até mesmo IDEs e editores de texto permitem a busca de texto usando expressões regulares como VSCode, PyCharm e Notepad++. Elas são tão úteis que o Pandas as torna a convenção de padrões padrão para muitos dos métodos de string mencionados anteriormente.

Aprenderemos apenas o suficiente sobre expressões regulares para concluir este caderno.

> Você pode consultar a documentação do Python sobre o pacote `re` aqui: https://docs.python.org/3/library/re.html. Para um tutorial mais completo sobre expressões regulares, confira meu artigo com a O'Reilly: https://www.oreilly.com/content/an-introduction-to-regular-expressions/

Vamos primeiro usar a biblioteca `re` do Python, que implementa expressões regulares. Vamos testar nossas expressões regulares com a função `fullmatch()` e envolvê-la em uma função chamada `regex_match()`, que simplesmente imprimirá se o padrão corresponde à string. Ela também fará uma formatação conveniente da cor da fonte na saída.

In [ ]:
import re

def red(str): 
    return '\033[91m' + str + '\033[0m'

def green(str): 
    return '\033[92m' + str + '\033[0m'

def regex_match(string, pattern):
    result = re.fullmatch(pattern=pattern, string=string)

    if result:
        print(f"{green(string)} Matches {green(pattern)}")
    else:
        print(f"{red(string)} Doesn't Match {red(pattern)}")

Para corresponder a um único caractere alfabético maiúsculo, use o intervalo de caracteres `[A-Z]` como espaço reservado para um único caractere. Observe como ele diferencia maiúsculas de minúsculas e você também pode definir intervalos arbitrários de letras.

In [ ]:
regex_match("C", "[A-Z]") # Match
regex_match("F", "[A-C]") # Doesn't Match
regex_match("3", "[A-Z]") # Doesn't Match 
regex_match("c", "[A-Z]") # Doesn't Match 
regex_match("-", "[A-Z]") # Doesn't Match 

Para corresponder letras maiúsculas e minúsculas, use `[A-Za-z]`.

In [ ]:
regex_match("C", "[A-ZA-z]") # Match
regex_match("c", "[A-Za-z]") # Matches
regex_match("3", "[A-Za-z]") # Doesn't Match 

Também podemos usar `[0-9]` para especificar um dígito válido de 0 a 9, ou qualquer intervalo arbitrário de um único dígito.

In [ ]:
regex_match("9", "[0-9]") # Match
regex_match("c", "[A-Za-z0-9]") # Match
regex_match("9", "[3-6]") # Doesn't Match
regex_match("C", "[0-9]") # Doesn't Match

Você também pode especificar um conjunto de letras, dígitos e caracteres. Abaixo, qualificamos apenas os caracteres A, C, F, 2, 8 ou 9.

In [ ]:
regex_match("9", "[ACF289]") # Match
regex_match("C", "[ACF289]") # Match
regex_match("7", "[ACF289]") # Doesn't Match
regex_match("G", "[ACF289]") # Doesn't Match

Letras e dígitos fora de um intervalo de caracteres `[ ]` são tratados literalmente como letras e dígitos em expressões regulares. Eles corresponderão apenas a esses valores.

In [ ]:
regex_match("Texas", "Texas") # Match
regex_match("Texas", "Arizona") # Doesn't Match 
regex_match("Texas", "TEXAS") # Doesn't Match 

Se quiser corresponder 3 letras maiúsculas do alfabeto, escreva `[A-Z]` três vezes ou coloque `{3}` repetições ao lado do intervalo de caracteres. Você também pode usar `{2,3}` para especificar um mínimo de 2 repetições e um máximo de `3`.

In [ ]:
regex_match("AEH", "[A-Z][A-Z][A-Z]") # Match
regex_match("AFH", "[A-Z]{3}") # Match
regex_match("AFH", "[A-Z]{2,3}") # Match
regex_match("AF", "[A-Z]{2,3}") # Match
regex_match("A9H", "[A-Z]{2,3}") # Doesn't Match

Se quiser corresponder a uma ou mais instâncias de um padrão, coloque um `+` ao lado dele. Por exemplo, `[A-Z]+` corresponderá a uma ou mais letras maiúsculas do alfabeto.

In [ ]:
regex_match("AEH", "[A-Z]+") # Match
regex_match("AEHSDHHHNHEHHBV", "[A-Z]+") # Match
regex_match("93572", "[0-9]+") # Match
regex_match("AEHSDHHHNHEHHBV", "[A-Z0-9]+") # Match
regex_match("93572", "[A-Z]+") # Doesn't Match
regex_match("AEHSDHHHNHEHHBV", "[0-9]+") # Doesn't Match

Outro quantificador útil é o `?`, que corresponde a 0 ou 1 instância de um padrão. Por exemplo, podemos usá-lo para especificar um dígito opcional antes de duas letras maiúsculas.

In [ ]:
regex_match("2GH", "[0-9]?[A-Z]{2}") # Match
regex_match("GH", "[0-9]?[A-Z]{2}") # Match
regex_match("2H", "[0-9]?[A-Z]{2}") # No Match
regex_match("22H", "[0-9]?[A-Z]{2}") # No Match

O ponto `.` representa um caractere curinga, correspondendo a qualquer caractere único, incluindo caracteres não alfanuméricos, como pontuação e símbolos. Se você pretende corresponder a um ponto literal, use uma barra de escape antes dele, `\.`.

Com um caractere curinga, você também pode colocar um quantificador como `{3}` ou `+` depois dele para especificar 3 caracteres ou um ou mais caracteres, respectivamente.

In [ ]:
regex_match("A#H", "...") # Match
regex_match("A#H", ".{3}") # Match 
regex_match("A#H", ".+") # Match
regex_match("AH", ".{3}") # Doesn't Match

Por fim, o último operador que precisamos conhecer é o agrupamento de parênteses `()`, bem como o alternador `|`. Se eu quiser corresponder apenas conexões de aeroporto de `ABQ` ou `DAL` a `HOU` ou `PHX`, posso expressar isso com `(ABQ|DAL)-(HOU|PHX)`.

In [ ]:
regex_match("ABQ", "(ABQ|DAL)") # Match 
regex_match("ABQ-HOU", "(ABQ|DAL)-(HOU|PHX)") # Match 
regex_match("DAL-HOU", "(ABQ|DAL)-(HOU|PHX)") # Match 
regex_match("DAL-PHX", "(ABQ|DAL)-(HOU|PHX)") # Match 
regex_match("PHX-DAL", "(ABQ|DAL)-(HOU|PHX)") # Doesn't Match 
regex_match("MDW-DAL", "(ABQ|DAL)-(HOU|PHX)") # Doesn't Match 


## Correspondências parciais de strings

Digamos que queremos encontrar todos os registros com um `Email` contendo o domínio `outlook.com`. Isso é bastante fácil usando a função `contains()` na propriedade `str`. Observe que a string padrão é tratada como uma expressão regular, portanto, precisamos escapar o ponto `.` com uma barra invertida `\.`. Caso contrário, ele será tratado como um curinga.

In [ ]:
df['Email'].str.contains('outlook\.com', regex=True)

Como um dos valores para e-mail é `NaN`, precisaremos tratá-lo se quisermos usá-lo como máscara de filtragem. Podemos fazer isso passando `na = False` para a função `contains()`. Isso fará com que os valores ausentes sejam tratados como `False`.

In [ ]:
df[df['Email'].str.contains('outlook\.com', regex=True, na=False)]

## Correspondências de sequência completa

Digamos que queremos rastrear endereços IP inválidos. Embora possamos [ser extremamente específicos e elaborados com padrões IPv4](https://stackoverflow.com/questions/5284147/validating-ipv4-addresses-with-regexp), vamos simplificar.

Abaixo está uma expressão regular simplista para corresponder a um endereço IP. Usamos a função `fullmatch()` para qualificar a string do endereço IP por completo.

In [ ]:
ipAddressRegex = r'[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}'

df['IP_ADDRESS'].str.fullmatch(ipAddressRegex)

> Normalmente, você só precisa tornar sua expressão regular específica o suficiente para capturar o que você está procurando nos dados. Se você não conhece bem seus dados, será melhor ser mais específico.

Vamos usar para qualificar endereços IP que não correspondem a uma condição. De fato, temos um endereço IP quebrado que excede 4 dígitos entre os separadores `.`.

In [ ]:
df[df['IP_ADDRESS'].str.fullmatch(ipAddressRegex) == False]

Aqui está outro exemplo de como encontrar números de telefone inválidos nos EUA. Observe como qualificamos os 3 primeiros dígitos, depois os 3 seguintes e, por fim, os 4 dígitos finais. Variantes que podem ou não conter hífens `-`, parênteses para o código de área `( )` e espaços. Com certeza, encontramos três números de telefone corrompidos.

In [ ]:
df[df['Phone'].str.fullmatch(r"\(?[0-9]{3}\)?[ -]?[0-9]{3}[ -]?[0-9]{4}") == False]

Vamos em frente e incluir apenas linhas em nosso dataframe que tenham números de telefone e endereços IP válidos.

In [ ]:
df = df[df['Phone'].str.fullmatch(r"\(?[0-9]{3}\)?[ -]?[0-9]{3}[ -]?[0-9]{4}")]

df = df[df['IP_ADDRESS'].str.fullmatch(ipAddressRegex)]

df

Por fim, vamos identificar todos os endereços de e-mail inválidos. Um e-mail precisa ter uma série de caracteres alfanuméricos (com alguns símbolos permitidos, como o ponto `.`), seguidos do símbolo `@` e, em seguida, do domínio. Também trataremos `na` como falso para capturar endereços de e-mail ausentes.

In [ ]:
df[df['Email'].str.fullmatch(r'[.A-Za-z0-9]+@[A-Za-z0-9]+\.[A-Za-z]+', na=False) == False]

Então, encontramos dois endereços de e-mail ausentes ou corrompidos. O e-mail da Lily não possui um domínio! Removeremos essas duas instâncias do dataframe.

In [ ]:
df = df[df['Email'].str.fullmatch(r'[.A-Za-z0-9]+@[A-Za-z0-9]+\.[A-Za-z]+', na=False)]

df

## Encontrando todas as correspondências

Também podemos usar `findall()` para procurar todas as correspondências parciais de uma expressão regular e retorná-las como uma série. Abaixo, extraímos todos os domínios de e-mail da coluna `Email`.

In [ ]:
df['Email'].str.findall(r'[A-Za-z0-9]+\.[A-Za-z]{3}$')

Se quisermos reunir os domínios exclusivos, podemos unir as "listas" de itens individuais em uma string e então qualificar os valores exclusivos.

In [ ]:
df['Email'].str.findall(r'[A-Za-z0-9]+\.[A-Za-z]{3}$').str.join("").unique()

## Substituindo Correspondências

Digamos que queremos limpar números de telefone removendo quaisquer traços `-`, parênteses `()` e espaços ` `. Podemos fazer isso usando um conjunto de caracteres de expressão regular `[-()]`. Observe que precisamos tornar o traço `-` o primeiro caractere para que ele não seja confundido com um operador de intervalo. Também adicionamos um espaço `` para capturar espaços.

In [ ]:
df['Phone'].str.replace(r"[- ()]", "", regex=True)

## Dividindo Texto

Uma ferramenta poderosa que podemos usar para dividir texto em colunas é a função `str.split()`. Fornecemos um padrão que pode ser um separador (como vírgulas `,`) ou um padrão de expressão regular completo.

Veja como podemos separar os domínios de e-mail em colunas separadas. Podemos então renomear essas colunas e adicioná-las de volta ao nosso dataframe.

In [ ]:
df['Email'].str.split("@", expand=True, regex=False)

Ao usar recursos de expressões regulares, como previsões, você obtém recursos de divisão mais poderosos com base nos caracteres adjacentes. Isso está além do escopo deste notebook.

## Exercício

Complete o código abaixo substituindo o ponto de interrogação `?`. Substitua-o por uma operação de expressão regular para identificar registros sem um número de rua no dataframe.

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "CUSTOMER_NAME" : ["Rex Tooling", "Prairie Construction", "Banke Logistics"],
    "STREET_ADDRESS" : ["147 Collie Way", "56 Samson Dr", "Elijah Blvd"]
})

df[? == False]

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "CUSTOMER_NAME" : ["Rex Tooling", "Prairie Construction", "Banke Logistics"],
    "STREET_ADDRESS" : ["147 Collie Way", "56 Samson Dr", "Elijah Blvd"]
})

df[df["STREET_ADDRESS"].str.fullmatch("[0-9]+ [A-Za-z0-9]+ (Way|Blvd|Dr|St)") == False]